<a href="https://colab.research.google.com/github/BroccoliWarrior/transformer-basic-knowledge/blob/main/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Embedding:token->vector***

将离散的vocab映射到连续的向量空间，使得语义接近的词在向量空间彼此接近

分布词表示

***Word2Vec***

input/output:one-hot

架构：

1.CBOW：连续词袋模型

通过上下文预测中心词，适合大型语料

2.Skip-gram：

通过中心词预测上下文，适合小型数据集

能够捕获词之间的语义关系

***BERT***

基于Transformer架构，采用双向编码器，可以更深入的理解上下文信息

词嵌入共享

1.what

核心思想是让模型中的不同部分或不同的任务，共同使用同一套词向量（Word Embeddings）

当模型需要将文本数据转换为数值向量时，它不再为每个任务或每个层创建独立的词向量，而是让它们共用同一个“词典”

2.how

在模型初始化时定义一个 nn.Embedding 层，然后让需要使用词向量的所有模块都引用这同一个层

In [ ]:
import torch
import torch.nn as nn

class SharedEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SharedEmbeddingModel, self).__init__()
        # 定义一个共享的词嵌入层
        self.shared_embedding = nn.Embedding(vocab_size, embedding_dim)

        # 编码器和解码器都使用这个共享的层
        self.encoder = MyEncoder(self.shared_embedding)
        self.decoder = MyDecoder(self.shared_embedding)

    def forward(self, input_data):
        # 编码器和解码器共享同一个词向量矩阵
        encoded_output = self.encoder(input_data)
        decoded_output = self.decoder(encoded_output)
        return decoded_output

class MyEncoder(nn.Module):
    def __init__(self, embedding_layer):
        super(MyEncoder, self).__init__()
        # 编码器直接接收共享的词嵌入层
        self.embedding = embedding_layer
        # ... 其他编码器层

class MyDecoder(nn.Module):
    def __init__(self, embedding_layer):
        super(MyDecoder, self).__init__()
        # 解码器也接收共享的词嵌入层
        self.embedding = embedding_layer
        # ... 其他解码器层


3.why

* 减少模型参数： 词向量矩阵是模型中参数量最大的部分之一。通过共享，可以显著减少模型的总参数量，降低内存占用，并加速训练。

* 增强泛化能力： 当不同任务共享词嵌入时，它们可以从彼此的学习中受益，共同为词语构建一个更全面、更鲁棒的表示。这有助于模型更好地理解语言，尤其是在数据量较少的任务上。

* 缓解数据稀疏性： 对于一些不常见的词，可能在一个任务中很少出现，但在另一个任务中相对频繁。共享词嵌入可以利用所有任务的数据来学习这些词的表示，从而缓解数据稀疏性问题。

4.problem

* “一个词，多个义”的问题： 共享词嵌入强制模型为同一个词学习一个单一的表示。但在某些任务中，一个词在不同上下文中可能有完全不同的含义（例如，机器翻译中，“Apple”既可以指公司也可以指水果）。共享词嵌入可能无法捕捉这种细微差别，从而限制了模型的表达能力。

* 负迁移学习： 如果两个任务之间的相关性很低，共享词嵌入可能会导致一个任务的学习过程干扰另一个任务，从而损害整体性能。

5.optimization

* 多头注意力（Multi-head Attention）的演变： 在Transformer模型中，自注意力机制的兴起允许模型根据上下文动态地为每个词生成不同的表示，而不是依赖于固定的词向量。

* 动态词嵌入： 诸如 ELMo、BERT 和 GPT 这样的预训练模型，彻底改变了词嵌入的概念。它们不再使用固定的词向量表，而是根据词语在句子中的具体上下文，动态地生成一个词向量。这种“上下文相关”的词嵌入从根本上解决了“一词多义”的问题，也让共享词嵌入变得不再那么必要，因为每个任务都可以通过微调来获得最适合其语境的词表示。